In [ ]:
from typing import List, Tuple, TypedDict, Type, Annotated, Sequence
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages

class AgentState(TypedDict):
    """ State of the agent."""
    messages: Annotated[Sequence[BaseMessage], add_messages]
    number_of_steps: int



### tool    
from langchain_core.tools import tool
from geopy.geocder import Nominatim 
from pydantic import BaseModel, Field
import requests

geolocator = Nominatim(user_agent="weather-app")

class SearchInput(BaseModel):
    location: str = Field(description="The city and the state,e.g. San Francisco, CA")
    date:str = Field(description="the forecating date for when to get the weather format(yyyy-mm-dd)")


@tool("get_weather_forecast",args_schema=SearchInput, return_direct=True)
def get_weather_forecast(location: str, date: str):
    """Retrieves the weather using Open-Meteo API for a given location (city) and a date (yyyy-mm-dd). Returns a list dictionary with the time and temperature for each hour."""
    location = geolocator.geocode(location)
    if location:
        try:
            response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={location.latitude}&longitude={location.longitude}&hourly=temperature_2m&start_date={date}&end_date={date}")         
            data = response.json()
            return 